In [1]:
import tensorflow as tf
import os
import imageio as imd
import numpy as np
import matplotlib.pyplot as plt
import random 
print(tf.__version__)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
def load_ml_data(data_directory):
        dirs = [d for d in os.listdir(data_directory)
               if os.path.isdir(os.path.join(data_directory, d))]
        print(dirs)
        labels = []
        images = []
        for d in dirs:
            label_dir = os.path.join(data_directory, d)
            file_names= [os.path.join(label_dir, f)
                          for f in os.listdir(label_dir)
                          if f.endswith(".ppm")]
            print(label_dir)
            print(file_names)
            
            for f in file_names:
                images.append(imd.imread(f))
                labels.append(int(d))
        return images,labels

In [ ]:
main_dir ='RedesNeuronales/'

train_data_dir = os.path.join(main_dir,"Training")
test_data_dir = os.path.join(main_dir,"Testing")

In [ ]:
images, labels = load_ml_data(train_data_dir)

In [ ]:
images = np.array(images)
labels = np.array(labels)
type(images)

In [ ]:
plt.hist(labels, len (set(labels)))
plt.show()

In [ ]:
rand_signs = random.sample(range(0,len(labels)), 10)
rand_signs 

In [ ]:
for i in range(len(rand_signs)):
    temp_im = images[rand_signs[i]]
    plt.subplot(1,10,i+1)
    plt.axis("off")
    plt.imshow(temp_im)
    plt.subplots_adjust(wspace= 0.5)
    plt.show()
    print("Forma:{0},min:{1}, max{2}".format(temp_im.shape, temp_im.min(),temp_im.max()))

In [ ]:
unique_labels=set(labels)
plt.figure(figsize=(16,16))
i=1
for label in unique_labels:
    temp_im=images[list(labels).index(label)]
    plt.subplot(8,8,i)
    plt.axis("off")
    plt.title("class {0} ({1})".format(label, list(labels).count(label)))
    i+=1
    plt.imshow(temp_im)
    
plt.show()

In [ ]:
from skimage import transform 
w=9999
h= 9999
for image in images:
    if image.shape[0] < h:
        h = image.shape[0]
        if image.shape[1] < w:
            w= image.shape[1]
print("Tamaño minimo: {0} x {1}".format(h,w))

In [ ]:
images30 = [transform.resize(image, (30,30)) for image in images ]

In [ ]:
images30[0]

In [ ]:
rand_signs = random.sample(range(0,len(labels)),10)
for i in range(len(rand_signs)):
    temp_im = images30[rand_signs[i]]
    plt.subplot(1,10,i+1)
    plt.axis("off")
    plt.imshow(temp_im)
    plt.subplots_adjust(wspace=0.5)
    plt.show()
    print("Forma:{0},min:{1}, max{2}".format(temp_im.shape, temp_im.min(),temp_im.max()))

In [ ]:
from skimage.color import rgb2gray
images30 = np.array(images30)
images30= rgb2gray(images30)


# Definir Red Neuronal

In [ ]:
x= tf.placeholder(dtype = tf.float32, shape=[None,30,30])

y= tf.placeholder(dtype=tf.int32, shape=[None])

images_flat = tf.contrib.layers.flatten(x)

logits = tf.contrib.layers.fully_connected(images_flat, 62, tf.nn.relu)

loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels =y, logits=logits))

train_opt = tf.train.AdamOptimizer(learning_rate = 0.001).minimize(loss)

final_pred = tf.argmax(logits, 1)

accuracy = tf.reduce_mean(tf.cast(final_pred, tf.float32))

In [ ]:
images_flat
logits 

# Entrenamiento de la Red Nuronal

In [ ]:
tf.set_random_seed(1234)

sess = tf.Session()

sess.run(tf.global_variables_initializer())

for i in range(1501):
    _,accuracy_val=sess.run([train_opt, accuracy], feed_dict={
                           x: images30,
                           y:list(labels)}
                           )
    if i%50 ==0:
        print("Eficacia:",accuracy_val)

# Evaluación de la red neuronal

In [ ]:
sample_idx = random.sample(range(len(images30)), 30)
sample_images = [images30[i] for i in sample_idx]
sample_labels = [labels[i] for i in sample_idx]

In [ ]:
prediction = sess.run([final_pred], feed_dict={x:sample_images}) [0]

In [ ]:
prediction

In [ ]:
sample_labels

In [ ]:
plt.figure(figsize=(16,20))
for i in range(len(sample_images)):
    real = sample_labels[i]
    prediccion = prediction[i]
    plt.subplot(10,4,i+1)
    plt.axis("off")
    color="blue" if real == prediccion else "red"
    plt.text(32,15, "Real:{0}\nPrediccion:{1}".format(real,prediccion),
            fontsize=14, color=color)
    plt.imshow(sample_images[i],cmap="gray")
plt.show()